In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
BOOK_SIZE = 20_000_000  # $20 Million (from Step 4)
DECAY = 3               # From the example logic
DELAY = 1               # Standard Delay-1

# --- STEP 0: GENERATE DUMMY MARKET DATA (The Matrix) ---
# We create a universe of 8 stocks over 10 days
np.random.seed(42)
dates = pd.date_range(start="2024-01-01", periods=10, freq="B")
stocks = [f"Stock_{i+1}" for i in range(8)]

# Create random returns (The "Input Matrix")
returns_df = pd.DataFrame(
    np.random.normal(0, 0.02, size=(len(dates), len(stocks))), 
    index=dates, 
    columns=stocks
)

print("📊 INPUT DATA (Returns Matrix - First 3 Days):")
print(returns_df.head(3))
print("-" * 50)

# --- STEP 1: EVALUATE ALPHA EXPRESSION ---
# Expression: rank(-returns)
# Logic: Rank returns from 0.0 to 1.0. Lowest return gets highest rank.
# Note: BRAIN rank scales from 0 to 1.
raw_alpha = (-1 * returns_df).rank(axis=1, pct=True)

# --- SIMULATION LOOP (Replicating Steps 2-7) ---
# We need to store the "Allocated Weights" to calculate PnL later
final_weights = pd.DataFrame(index=dates, columns=stocks)
previous_day_weights = pd.Series(0, index=stocks) # Start with 0 weights

print("\n⚙️ RUNNING SIMULATION STEPS...")

for date in dates:
    # Get today's raw alpha vector
    alpha_vector = raw_alpha.loc[date]
    
    # --- DECAY CALCULATION (From Text) ---
    # Formula: (Today + Decay * Yesterday_Final) / (Decay + 1)
    # Note: This happens BEFORE neutralization in some engines, 
    # but the text implies it smooths the final output. 
    # We apply it to the raw vector here for consistency with standard smoothing.
    decayed_vector = (alpha_vector + (DECAY * previous_day_weights)) / (DECAY + 1)
    
    # --- STEP 2: NEUTRALIZATION ---
    # Subtract the average so sum is 0
    neutralized_vector = decayed_vector - decayed_vector.mean()
    
    # --- STEP 3: NORMALIZATION ---
    # Scale so absolute sum is 1.0
    abs_sum = neutralized_vector.abs().sum()
    if abs_sum == 0:
        normalized_weights = neutralized_vector * 0 # Handle divide by zero
    else:
        normalized_weights = neutralized_vector / abs_sum
        
    # Store these weights (These are the weights we want to hold TODAY)
    final_weights.loc[date] = normalized_weights
    
    # Update previous day for next loop's decay calculation
    previous_day_weights = normalized_weights

# --- STEP 4 & 5: ALLOCATION & PNL ---
# We apply the weights to the Book Size
position_dollars = final_weights * BOOK_SIZE

# PnL Calculation (Delay 1)
# We earn returns tomorrow based on the positions we chose today.
# So we shift positions forward by 1 day.
held_positions = position_dollars.shift(1).fillna(0)
daily_pnl = (held_positions * returns_df).sum(axis=1)

# --- STEP 7: CUMULATIVE PNL CHART ---
cumulative_pnl = daily_pnl.cumsum()

print("\n💰 SIMULATION RESULTS:")
print(f"Total PnL: ${cumulative_pnl.iloc[-1]:,.2f}")

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(cumulative_pnl, label='Alpha PnL', marker='o')
plt.title(f"Cumulative PnL (Decay {DECAY}, Book ${BOOK_SIZE/1e6:,.0f}M)")
plt.ylabel("PnL ($)")
plt.grid(True)
plt.show()

# Show the internals for the user to debug
print("\n🔍 DETAILED INTERNALS (Last Day):")
debug_df = pd.DataFrame({
    "Return": returns_df.iloc[-1],
    "Raw_Alpha": raw_alpha.iloc[-1],
    "Final_Weight": final_weights.iloc[-1],
    "Position ($)": position_dollars.iloc[-1]
})
print(debug_df)